In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import io
import time
import os
import json
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache

class StockAnalyzer:

    def __init__(self):
            self.base_dir = 'stock_analysis'
            self.stock_lists_dir = os.path.join(self.base_dir, 'stock_lists')
            self.results_dir = os.path.join(self.base_dir, 'daily_results')
            self.delisted_cache_file = os.path.join(self.stock_lists_dir, 'delisted_stocks.json')
            self.setup_directories()
            self.load_delisted_stocks()
            self.session = requests.Session()  # Reuse session for better performance

    def setup_directories(self):
        """Create necessary directories if they don't exist"""
        os.makedirs(self.stock_lists_dir, exist_ok=True)
        os.makedirs(self.results_dir, exist_ok=True)
        
    def should_update_stock_list(self, exchange):
        """Check if stock list needs updating (weekly update)"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        if not os.path.exists(list_file):
            return True
        # Check if file is older than 7 days
        file_time = datetime.fromtimestamp(os.path.getmtime(list_file))
        return (datetime.now() - file_time).days >= 7
    
    def load_delisted_stocks(self):
        """Load previously identified delisted stocks"""
        try:
            with open(self.delisted_cache_file, 'r') as f:
                self.delisted_stocks = json.load(f)
        except FileNotFoundError:
            self.delisted_stocks = {'NSE': [], 'BSE': []}  # Changed from set to list
            self.save_delisted_stocks()

    def save_delisted_stocks(self):
        """Save delisted stocks to cache"""
        with open(self.delisted_cache_file, 'w') as f:
            json.dump({
                'NSE': self.delisted_stocks['NSE'],
                'BSE': self.delisted_stocks['BSE']
            }, f)
    
    def save_stock_list(self, symbols, exchange):
        """Save stock list to JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        data = {
            'symbols': symbols,
            'last_updated': datetime.now().strftime('%Y-%m-%d')
        }
        with open(list_file, 'w') as f:
            json.dump(data, f)

    def load_stock_list(self, exchange):
        """Load stock list from JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        with open(list_file, 'r') as f:
            data = json.load(f)
        return data['symbols']
    
    def download_nse_stocks(self):
        """Download NSE stock list if needed"""
        if self.should_update_stock_list('NSE'):
            try:
                url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
                response = requests.get(url)
                df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
                symbols = df['SYMBOL'].tolist()
                self.save_stock_list(symbols, 'NSE')
                print("NSE stock list updated")
                return symbols
            except Exception as e:
                print(f"Error downloading NSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'NSE_stocks.json')):
                    return self.load_stock_list('NSE')
                return []
        return self.load_stock_list('NSE')

    def download_bse_stocks(self):
        """Download BSE stock list if needed"""
        if self.should_update_stock_list('BSE'):
            try:
                # Using the BSE list from static file or an appropriate source
                url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                table = soup.find('table', {'id': 'ContentPlaceHolder1_tblData'})
                symbols = []

                if table:
                    rows = table.find_all('tr')[1:]  # Skip header row
                    for row in rows:
                        cols = row.find_all('td')
                        if cols:
                            bse_code = cols[0].text.strip()
                            symbols.append(bse_code)

                if symbols:
                    self.save_stock_list(symbols, 'BSE')
                    print("BSE stock list updated")
                    return symbols
            except Exception as e:
                print(f"Error downloading BSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'BSE_stocks.json')):
                    return self.load_stock_list('BSE')
                return []
        return self.load_stock_list('BSE')

    @lru_cache(maxsize=1000)
    def calculate_rsi(self, prices_tuple, periods=14):
        """Calculate RSI with caching for performance"""
        prices = pd.Series(prices_tuple)
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=periods).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
        rs = gain / loss
        return float(100 - (100 / (1 + rs)).iloc[-1])

    def get_stock_data(self, symbol, exchange='NSE', days=60):
        """Fetch detailed stock data with additional information"""
        if symbol in self.delisted_stocks[exchange]:
            return None, None, None, None

        try:
            ticker_symbol = f"{symbol}.NS" if exchange == 'NSE' else f"{symbol}.BO"
            ticker = yf.Ticker(ticker_symbol)
            
            # Get stock info
            info = ticker.info
            
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)

            hist_data = ticker.history(
                start=start_date,
                end=end_date,
                interval="1d"
            )

            if hist_data.empty or len(hist_data) < 20:
                self.delisted_stocks[exchange].append(symbol)  # Changed from add to append
                self.save_delisted_stocks()  # Save immediately after adding
                return None, None, None, None

            latest_price = hist_data['Close'].iloc[-1]
            prices_tuple = tuple(hist_data['Close'].values)
            latest_rsi = self.calculate_rsi(prices_tuple)

            return latest_price, hist_data, latest_rsi, info

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            self.delisted_stocks[exchange].append(symbol)  # Changed from add to append
            self.save_delisted_stocks()  # Save immediately after adding
            return None, None, None, None

    def process_stock_batch(self, batch_data):
        """Process a batch of stocks with enhanced information"""
        symbols, exchange = batch_data
        results = []
        
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = {executor.submit(self.get_stock_data, symbol, exchange): symbol 
                      for symbol in symbols}
            
            for future in futures:
                symbol = futures[future]
                try:
                    latest_price, hist_data, latest_rsi, info = future.result()
                    if all(v is not None for v in [latest_price, hist_data, latest_rsi, info]):
                        # Extract additional information safely using get() method
                        stock_info = {
                            'Symbol': symbol,
                            'Exchange': exchange,
                            'Company_Name': info.get('longName', 'N/A'),
                            'Latest_Price': latest_price,
                            'RSI': latest_rsi,
                            'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 0,
                            'Market_Cap': info.get('marketCap', 'N/A'),
                            'PE_Ratio': info.get('trailingPE', 'N/A'),
                            'EPS': info.get('trailingEps', 'N/A'),
                            'Dividend_Yield': info.get('dividendYield', 'N/A'),
                            'Book_Value': info.get('bookValue', 'N/A'),
                            'Sector': info.get('sector', 'N/A'),
                            'Industry': info.get('industry', 'N/A'),
                            '52W_High': info.get('fiftyTwoWeekHigh', 'N/A'),
                            '52W_Low': info.get('fiftyTwoWeekLow', 'N/A'),
                            '50d_MA': info.get('fiftyDayAverage', 'N/A'),
                            '200d_MA': info.get('twoHundredDayAverage', 'N/A'),
                            'Beta': info.get('beta', 'N/A'),
                            'Previous_Close': info.get('previousClose', 'N/A'),
                            'Open': info.get('open', 'N/A'),
                            'Day_High': info.get('dayHigh', 'N/A'),
                            'Day_Low': info.get('dayLow', 'N/A'),
                            'Date': datetime.now().strftime('%Y-%m-%d')
                        }
                        
                        # Calculate additional metrics
                        if hist_data is not None:
                            stock_info.update({
                                'Volatility_30d': hist_data['Close'].pct_change().std() * np.sqrt(252),
                                'Returns_30d': (hist_data['Close'].iloc[-1] / hist_data['Close'].iloc[0] - 1) * 100,
                                'Average_Volume_30d': hist_data['Volume'].mean()
                            })
                        
                        results.append(stock_info)
                        
                except Exception as e:
                    print(f"Error processing {symbol}: {e}")

        return results

    def run_daily_analysis(self):
        """Run daily analysis with Excel output"""
        current_date = datetime.now().strftime('%Y%m%d')
        
        print("Loading stock lists...")
        nse_symbols = self.download_nse_stocks()
        bse_symbols = self.download_bse_stocks()

        print(f"\nAnalyzing {len(nse_symbols)} NSE and {len(bse_symbols)} BSE stocks")

        for exchange, symbols in [('NSE', nse_symbols), ('BSE', bse_symbols)]:
            batch_data=(symbols, exchange)
            results = self.process_stock_batch(batch_data)
            
            if results:
                # Create Excel writer with xlsxwriter engine
                excel_file = os.path.join(
                    self.results_dir,
                    f'{exchange.lower()}_analysis_{current_date}.xlsx'
                )
                
                # Convert results list to DataFrame
                results_df = pd.DataFrame(results)
                
                with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                    # Write all stocks to first sheet
                    results_df.to_excel(writer, sheet_name='All Stocks', index=False)
                    
                    # Write high RSI stocks to second sheet
                    high_rsi = results_df[results_df['RSI'] >= 40].sort_values('RSI', ascending=False)
                    high_rsi.to_excel(writer, sheet_name='High RSI Stocks', index=False)
                    
                    # Get workbook and worksheet objects for formatting
                    workbook = writer.book
                    
                    # Add formats
                    header_format = workbook.add_format({
                        'bold': True,
                        'text_wrap': True,
                        'valign': 'top',
                        'bg_color': '#D9E1F2',
                        'border': 1
                    })
                    
                    # Format each worksheet
                    for worksheet in writer.sheets.values():
                        # Set column widths
                        worksheet.set_column('A:Z', 15)
                        # Apply header format to first row
                        for col_num, value in enumerate(results_df.columns.values):
                            worksheet.write(0, col_num, value, header_format)

                print(f"\n{exchange} analysis complete. Results saved to {excel_file}")
                
                # Print high RSI stocks
                print(f"\n{exchange} Stocks with RSI >= 40:")
                for _, row in high_rsi.iterrows():
                    print(colored(
                        f"{row['Symbol']} ({row['Company_Name']}): RSI = {row['RSI']:.2f}, "
                        f"Price = ₹{row['Latest_Price']:.2f}",
                        'green', attrs=['bold']
                    ))

        print(f"\nDaily analysis complete. Results saved in {self.results_dir}")

if __name__ == "__main__":
    analyzer = StockAnalyzer()
    analyzer.run_daily_analysis()

Loading stock lists...

Analyzing 2062 NSE and 3026 BSE stocks


DHAN-RE.NS: Period '1mo' is invalid, must be one of ['1d', '5d']
INDOFARM.NS: Period '1mo' is invalid, must be one of ['1d', '5d', 'ytd', 'max']
$KALYANI.NS: possibly delisted; no price data found  (1d 2024-11-10 13:15:54.587470 -> 2025-01-09 13:15:54.587470)
$MOKSH-RE.NS: possibly delisted; no price data found  (1d 2024-11-10 13:16:15.487805 -> 2025-01-09 13:16:15.487805)



NSE analysis complete. Results saved to stock_analysis\daily_results\nse_analysis_20250109.xlsx

NSE Stocks with RSI >= 40:
AARVEEDEN (Aarvee Denims and Exports Ltd.): RSI = 100.00, Price = ₹129.00
BGRENERGY (BGR Energy Systems Limited): RSI = 100.00, Price = ₹114.39
BLUECOAST (Blue Coast Hotels Limited): RSI = 100.00, Price = ₹21.84
VISASTEEL (VISA Steel Limited): RSI = 100.00, Price = ₹45.48
NIRAJISPAT (Niraj Ispat Industries Limited): RSI = 100.00, Price = ₹245.18
ATLASCYCLE (Atlas Cycles (Haryana) Limited): RSI = 100.00, Price = ₹102.56
FSC (Future Supply Chain Solutions Limited): RSI = 96.77, Price = ₹2.49
INDOTHAI (Indo Thai Securities Limited): RSI = 94.57, Price = ₹1648.95
PTCIL (PTC Industries Limited): RSI = 92.92, Price = ₹17424.75
SVLL (Shree Vasu Logistics Limited): RSI = 91.62, Price = ₹386.95
AXISCADES (AXISCADES Technologies Limited): RSI = 90.15, Price = ₹809.15
BROOKS (Brooks Laboratories Limited): RSI = 88.43, Price = ₹189.58
SHAKTIPUMP (Shakti Pumps (India) Limited

$500032.BO: possibly delisted; no timezone found
$500040.BO: possibly delisted; no timezone found
$500038.BO: possibly delisted; no timezone found
$500033.BO: possibly delisted; no timezone found
$500055.BO: possibly delisted; no price data found  (1d 2024-11-10 13:17:44.847840 -> 2025-01-09 13:17:44.847840)
$500042.BO: possibly delisted; no timezone found
$500052.BO: possibly delisted; no timezone found
$500048.BO: possibly delisted; no timezone found
$500049.BO: possibly delisted; no timezone found
$500078.BO: possibly delisted; no timezone found
$500074.BO: possibly delisted; no timezone found
$500086.BO: possibly delisted; no timezone found
$500087.BO: possibly delisted; no timezone found
$500084.BO: possibly delisted; no timezone found
$500092.BO: possibly delisted; no timezone found
$500096.BO: possibly delisted; no timezone found
$500101.BO: possibly delisted; no timezone found
$500103.BO: possibly delisted; no timezone found
$500106.BO: possibly delisted; no timezone found
$500


BSE analysis complete. Results saved to stock_analysis\daily_results\bse_analysis_20250109.xlsx

BSE Stocks with RSI >= 40:
533499 (Bharatiya Global Infomedia Limited): RSI = 100.00, Price = ₹4.74
539266 (Concrete Infra & Media Limited): RSI = 100.00, Price = ₹3.99
531663 (Yuvraaj Hygiene Products Limited): RSI = 100.00, Price = ₹5.59
539032 (Fraser and Company Limited): RSI = 100.00, Price = ₹9.12
526905 (N/A): RSI = 100.00, Price = ₹13.74
513295 (N/A): RSI = 100.00, Price = ₹69.44
531769 (PFL Infotech Limited): RSI = 100.00, Price = ₹5.61
532090 (Vandana Knitwear Limited): RSI = 100.00, Price = ₹11.22
532368 (Brightcom Group Limited): RSI = 100.00, Price = ₹10.28
531502 (N/A): RSI = 100.00, Price = ₹6.73
514274 (Aarvee Denims and Exports Ltd.): RSI = 100.00, Price = ₹126.35
539544 (Abhishek Infraventures Limited): RSI = 100.00, Price = ₹7.83
534535 (Bio Green Papers Limited): RSI = 100.00, Price = ₹41.63
511401 (N/A): RSI = 100.00, Price = ₹20.27
514412 (Sarup Industries Limited): R